Second Version that using CIFAR-100 to train a Resnet-18

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import numpy as np
from google.colab import drive
import os
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report

In [2]:
print(">>> Environment Configurating...")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Use device: {device}")

drive.mount('/content/drive')
drive_root_path = '/content/drive/MyDrive/Datasets/cifar100_data'

if not os.path.exists(drive_root_path):
    os.makedirs(drive_root_path)

transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.507, 0.487, 0.441), (0.267, 0.256, 0.276)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.507, 0.487, 0.441), (0.267, 0.256, 0.276)),
])

>>> Environment Configurating...
Use device: cuda
Mounted at /content/drive


In [22]:
print(">>> Loading CIFAR-100 dataset...")
trainset = torchvision.datasets.CIFAR100(root=drive_root_path, train=True,
                                         download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR100(root=drive_root_path, train=False,
                                        download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100,
                                         shuffle=False, num_workers=2)

classes = trainset.classes

>>> Loading CIFAR-100 dataset...


In [4]:
def mixup_data(x, y, alpha=1.0):
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size()[0]
    index = torch.randperm(batch_size).to(device)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

In [5]:
print(">>> Building the ResNet-18 model (Modified for CIFAR-100)...")

def get_cifar_resnet18():
    net1 = models.resnet18(weights=None)
    net1.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
    net1.maxpool = nn.Identity()
    net1.fc = nn.Linear(net1.fc.in_features, 100)
    return net1

net1 = get_cifar_resnet18().to(device)

optimizer = optim.SGD(net1.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
total_epochs = 100
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=total_epochs)
criterion = nn.CrossEntropyLoss()

>>> Building the ResNet-18 model (Modified for CIFAR-100)...


In [7]:
print(f">>> Start training (Total Epochs: {total_epochs})...")
print("Strategy: Mixup Enabled | Optimizer: SGD | Scheduler: CosineAnnealing")

train_losses = []

for epoch in range(total_epochs):
    net1.train()
    running_loss = 0.0

    for i, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        inputs, targets_a, targets_b, lam = mixup_data(inputs, targets, alpha=1.0)
        optimizer.zero_grad()
        outputs = net1(inputs)
        loss = mixup_criterion(criterion, outputs, targets_a, targets_b, lam)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    scheduler.step()
    avg_loss = running_loss / len(trainloader)
    train_losses.append(avg_loss)
    current_lr = optimizer.param_groups[0]['lr']

    if (epoch + 1) % 10 == 0:
        print(f'[Epoch {epoch+1:03d}/{total_epochs}] Loss: {avg_loss:.4f} | LR: {current_lr:.6f}')
    elif (epoch + 1) == 1:
        print(f'[Epoch 001] Initial Loss: {avg_loss:.4f}')

print("Training completed!")
torch.save(net1.state_dict(), f'{drive_root_path}/cifar100_resnet18_scratch.pth')
print(f"The model has been saved to: {drive_root_path}/cifar100_resnet18_scratch.pth")

>>> Start training (Total Epochs: 100)...
Strategy: Mixup Enabled | Optimizer: SGD | Scheduler: CosineAnnealing
[Epoch 001] Initial Loss: 2.3236
[Epoch 010/100] Loss: 2.1636 | LR: 0.036050
[Epoch 020/100] Loss: 2.0820 | LR: 0.021896
[Epoch 030/100] Loss: 1.7998 | LR: 0.010492
[Epoch 040/100] Loss: 1.6527 | LR: 0.002956
[Epoch 050/100] Loss: 1.6067 | LR: 0.000025
[Epoch 060/100] Loss: 1.6467 | LR: 0.001985
[Epoch 070/100] Loss: 1.7260 | LR: 0.008646
[Epoch 080/100] Loss: 1.9693 | LR: 0.019355
[Epoch 090/100] Loss: 2.0941 | LR: 0.033063
[Epoch 100/100] Loss: 2.1906 | LR: 0.048429
Training completed!
The model has been saved to: /content/drive/MyDrive/Datasets/cifar100_data/cifar100_resnet18_scratch.pth


In [8]:
def evaluate_performance(model, dataloader, classes, target_class_name='bus'):
    model.eval()
    all_preds = []
    all_targets = []

    print(">>> The test set undergoing a comprehensive evaluation...")

    with torch.no_grad():
        for inputs, targets in dataloader:
            inputs = inputs.to(device)
            targets = targets.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())

    acc = accuracy_score(all_targets, all_preds)

    print("\n" + "="*40)
    print("Evaluation Report (Global Metrics)")
    print("="*40)
    print(f"Accuracy  : {acc:.2%}")

    report_dict = classification_report(all_targets, all_preds, target_names=classes, output_dict=True, zero_division=0)
    weighted = report_dict['weighted avg']
    print(f"Weighted F1 : {weighted['f1-score']:.2%}")

    print("\n" + "-"*40)
    print(f"Target Category '{target_class_name}' Performance")
    print("-"*40)

    if target_class_name in classes:
        stats = report_dict[target_class_name]
        print(f"Precision : {stats['precision']:.2%}")
        print(f"Recall    : {stats['recall']:.2%}")
        print(f"F1-Score  : {stats['f1-score']:.2%}")
    else:
        print(f"Category not found: {target_class_name}")

evaluate_performance(net1, testloader, classes, target_class_name='bus')

>>> The test set undergoing a comprehensive evaluation...

Evaluation Report (Global Metrics)
Accuracy  : 63.31%
Weighted F1 : 62.71%

----------------------------------------
Target Category 'bus' Performance
----------------------------------------
Precision : 58.10%
Recall    : 61.00%
F1-Score  : 59.51%


The New Version that use fine-tuning strategy on pretrained Resnet-18 instead of building & training from the ground

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import numpy as np
from google.colab import drive
import os
from PIL import Image
import matplotlib.pyplot as plt

print(">>> Setting up...")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

drive.mount('/content/drive')
drive_root_path = '/content/drive/MyDrive/Datasets/cifar100_data'

if not os.path.exists(drive_root_path):
    os.makedirs(drive_root_path)

expected_file = os.path.join(drive_root_path, 'cifar-100-python.tar.gz')
expected_folder = os.path.join(drive_root_path, 'cifar-100-python')

if os.path.exists(expected_file):
    print(f"zip detected: {expected_file}")
elif os.path.exists(expected_folder):
    print(f"folder detected: {expected_folder}")
else:
    print(f"file not found. Current path: {drive_root_path}")

try:
    trainset = torchvision.datasets.CIFAR100(root=drive_root_path, train=True, download=True)
    print("Success!")
except RuntimeError as e:
    print(f"Failed: {e}")

>>> Setting up...
Using device: cuda
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
zip detected: /content/drive/MyDrive/Datasets/cifar100_data/cifar-100-python.tar.gz
Success!


In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models

transform_transfer = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

print(">>> Loading CIFAR-100, resizing images to 224x224.")
trainset = torchvision.datasets.CIFAR100(root='./data', train=True,
                                        download=True, transform=transform_transfer)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR100(root='./data', train=False,
                                       download=True, transform=transform_transfer)
testloader = torch.utils.data.DataLoader(testset, batch_size=32,
                                         shuffle=False, num_workers=2)

print(">>> Downloading ImageNet pre-trained weights...")
net = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
num_ftrs = net.fc.in_features
net.fc = nn.Linear(num_ftrs, 100)
net = net.to(device)

optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss()

epochs = 5
print(f">>> Start fine-tuning for {epochs} epochs...")

for epoch in range(epochs):
    net.train()
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        if i % 100 == 0:
            print(f'[Epoch {epoch+1}, Batch {i}] Loss: {loss.item():.3f}')

    print(f'Epoch {epoch+1} Done. Avg Loss: {running_loss / len(trainloader):.3f}')

print("All Done!")

>>> Loading CIFAR-100, resizing images to 224x224.


100%|██████████| 169M/169M [00:13<00:00, 12.2MB/s]


>>> Downloading ImageNet pre-trained weights...
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 239MB/s]


>>> Start fine-tuning for 5 epochs...
[Epoch 1, Batch 0] Loss: 5.081
[Epoch 1, Batch 100] Loss: 4.260
[Epoch 1, Batch 200] Loss: 3.481
[Epoch 1, Batch 300] Loss: 2.721
[Epoch 1, Batch 400] Loss: 2.472
[Epoch 1, Batch 500] Loss: 1.947
[Epoch 1, Batch 600] Loss: 2.230
[Epoch 1, Batch 700] Loss: 1.763
[Epoch 1, Batch 800] Loss: 1.363
[Epoch 1, Batch 900] Loss: 1.151
[Epoch 1, Batch 1000] Loss: 1.016
[Epoch 1, Batch 1100] Loss: 1.142
[Epoch 1, Batch 1200] Loss: 1.205
[Epoch 1, Batch 1300] Loss: 1.062
[Epoch 1, Batch 1400] Loss: 0.870
[Epoch 1, Batch 1500] Loss: 0.935
Epoch 1 Done. Avg Loss: 1.999
[Epoch 2, Batch 0] Loss: 1.172
[Epoch 2, Batch 100] Loss: 0.984
[Epoch 2, Batch 200] Loss: 0.882
[Epoch 2, Batch 300] Loss: 0.817
[Epoch 2, Batch 400] Loss: 0.932
[Epoch 2, Batch 500] Loss: 1.684
[Epoch 2, Batch 600] Loss: 1.233
[Epoch 2, Batch 700] Loss: 1.081
[Epoch 2, Batch 800] Loss: 0.884
[Epoch 2, Batch 900] Loss: 0.693
[Epoch 2, Batch 1000] Loss: 0.799
[Epoch 2, Batch 1100] Loss: 0.783
[Epo

In [11]:
import torch
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

transform_eval = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

testset_eval = torchvision.datasets.CIFAR100(root='./data', train=False,
                                            download=True, transform=transform_eval)
testloader_eval = torch.utils.data.DataLoader(testset_eval, batch_size=32,
                                             shuffle=False, num_workers=2)

classes = testset_eval.classes

def evaluate_resnet_performance(model, dataloader):
    model.eval()
    all_preds = []
    all_targets = []

    print(f">>> Evaluating on CIFAR-100 test set...")
    print(f">>> This may take some time, processing 10,000 images of size 224x224")

    with torch.no_grad():
        for inputs, targets in dataloader:
            inputs = inputs.to(device)
            targets = targets.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())

    acc = accuracy_score(all_targets, all_preds)

    print("\n" + "="*50)
    print(f"Final Model Performance (ResNet-18 Fine-tuned)")
    print("="*50)
    print(f"Overall Accuracy: {acc:.2%}")
    print("-" * 50)

    report = classification_report(all_targets, all_preds, target_names=classes, output_dict=True)
    weighted_avg = report['weighted avg']
    print(f"Weighted Average Metrics:")
    print(f"Precision: {weighted_avg['precision']:.2%}")
    print(f"Recall:    {weighted_avg['recall']:.2%}")
    print(f"F1-Score:  {weighted_avg['f1-score']:.2%}")

    print("-" * 50)

    target_vehicles = ['bus', 'pickup_truck', 'train', 'truck', 'streetcar']
    print("Target Category Performance (CAPTCHA Task Focus):")
    print(f"{'Class Name':<15} | {'Precision':<10} | {'Recall':<10} | {'F1-Score':<10}")
    print("-" * 55)

    for class_name in target_vehicles:
        if class_name in report:
            stats = report[class_name]
            print(f"{class_name:<15} | {stats['precision']:.2%}    | {stats['recall']:.2%}    | {stats['f1-score']:.2%}")
        else:
            print(f"{class_name:<15} | Category not found")

    return acc

evaluate_resnet_performance(net, testloader_eval)

>>> Evaluating on CIFAR-100 test set...
>>> This may take some time, processing 10,000 images of size 224x224

Final Model Performance (ResNet-18 Fine-tuned)
Overall Accuracy: 80.29%
--------------------------------------------------
Weighted Average Metrics:
Precision: 80.83%
Recall:    80.29%
F1-Score:  80.24%
--------------------------------------------------
Target Category Performance (CAPTCHA Task Focus):
Class Name      | Precision  | Recall     | F1-Score  
-------------------------------------------------------
bus             | 77.78%    | 84.00%    | 80.77%
pickup_truck    | 92.08%    | 93.00%    | 92.54%
train           | 83.96%    | 89.00%    | 86.41%
truck           | Category not found
streetcar       | 78.64%    | 81.00%    | 79.80%


0.8029

Testing performace on self-prepared test set

In [12]:
!pip install icrawler

In [13]:
import os
import random
import shutil
import torch
import torchvision
from PIL import Image
from icrawler.builtin import BingImageCrawler
from google.colab import drive

print(">>> [1/5] Mounting Google Drive...")
drive.mount('/content/drive')

base_root = "/content/drive/MyDrive/Datasets/auto_test_set"
raw_root = os.path.join(base_root, "raw_temp")
final_root = os.path.join(base_root, "clean_224")

if os.path.exists(base_root):
    shutil.rmtree(base_root)
os.makedirs(raw_root, exist_ok=True)
os.makedirs(final_root, exist_ok=True)

print(">>> [2/5] Getting CIFAR-100 class list...")
temp_set = torchvision.datasets.CIFAR100(root='./temp_data', train=False, download=True)
cifar100_classes = temp_set.classes

selected_keywords = random.sample(cifar100_classes, 10)
print(f"Randomly selected 10 target classes: {selected_keywords}")

def download_images(keywords, max_num=20):
    for keyword in keywords:
        print(f"\nCrawling: {keyword} ...")
        search_term = keyword + " photo"
        save_dir = os.path.join(raw_root, keyword)
        os.makedirs(save_dir, exist_ok=True)

        crawler = BingImageCrawler(
            downloader_threads=4,
            storage={"root_dir": save_dir},
            log_level="ERROR"
        )
        crawler.crawl(keyword=search_term, max_num=max_num)

def process_images_for_model():
    print("\n>>> [4/5] Processing and converting images...")
    valid_exts = {".jpg", ".jpeg", ".png", ".bmp", ".webp"}
    count = 0

    for class_name in os.listdir(raw_root):
        class_dir = os.path.join(raw_root, class_name)
        if not os.path.isdir(class_dir): continue

        target_dir = os.path.join(final_root, class_name)
        os.makedirs(target_dir, exist_ok=True)

        for filename in os.listdir(class_dir):
            if not any(filename.lower().endswith(ext) for ext in valid_exts): continue

            src_path = os.path.join(class_dir, filename)

            try:
                with Image.open(src_path) as img:
                    img = img.convert("RGB")
                    img_resized = img.resize((224, 224), Image.BICUBIC)
                    new_filename = f"{class_name}_{count}.jpg"
                    img_resized.save(os.path.join(target_dir, new_filename), "JPEG", quality=95)
                    count += 1
            except Exception as e:
                print(f"Skipping corrupted image: {filename}")

    print(f"\nDataset creation completed!")
    print(f"Storage location: {final_root}")
    print(f"Total valid images: {count}")

download_images(selected_keywords, max_num=15)
process_images_for_model()

>>> [1/5] Mounting Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
>>> [2/5] Getting CIFAR-100 class list...


100%|██████████| 169M/169M [00:14<00:00, 11.9MB/s]


Randomly selected 10 target classes: ['sunflower', 'trout', 'couch', 'maple_tree', 'mouse', 'lamp', 'rocket', 'mushroom', 'streetcar', 'dolphin']

Crawling: sunflower ...


ERROR:downloader:Response status code 404, file https://i0.wp.com/dhcrop.bsmrau.net/wp-content/uploads/2018/01/Sunflower.jpg
ERROR:downloader:Exception caught when downloading file https://www.dbrl.org/wp-content/uploads/2023/04/sunflower-5539198_1920.jpg, error: HTTPSConnectionPool(host='www.dbrl.org', port=443): Max retries exceeded with url: /wp-content/uploads/2023/04/sunflower-5539198_1920.jpg (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7829aec0c5f0>, 'Connection to www.dbrl.org timed out. (connect timeout=5)')), remaining retry times: 2



Crawling: trout ...


...


Crawling: mushroom ...


ERROR:downloader:Response status code 403, file https://get.pxhere.com/photo/nature-forest-flower-autumn-mushroom-close-forest-floor-fungus-mushrooms-toxic-agaric-bolete-fungal-species-forest-mushroom-lamellar-matsutake-matryoshka-forest-plant-symbol-of-good-luck-screen-fungus-edible-mushroom-medicinal-mushroom-agaricaceae-penny-bun-603884.jpg
ERROR:downloader:Response status code 403, file https://get.pxhere.com/photo/nature-forest-meadow-prairie-flower-gift-red-autumn-mushroom-point-flora-fungus-spotted-fly-agaric-toxic-amanita-agaric-bolete-amanita-muscaria-muscaria-red-fly-agaric-mushroom-macro-photography-edible-mushroom-medicinal-mushroom-penny-bun-amanitaceae-1157240.jpg
ERROR:downloader:Response status code 403, file https://get.pxhere.com/photo/nature-forest-flower-autumn-botany-mushroom-flora-fauna-fungus-mushrooms-woodland-agaric-bolete-agaricus-matsutake-oyster-mushroom-edible-mushroom-medicinal-mushroom-agaricomycetes-agaricaceae-champignon-mushroom-penny-bun-pleurotus-ery


Crawling: streetcar ...

Crawling: dolphin ...


ERROR:downloader:Response status code 403, file https://c.pxhere.com/photos/2f/a4/ocean_nikon_dolphin_maldives_d4_nikond-412381.jpg
ERROR:downloader:Response status code 404, file http://upload.wikimedia.org/wikipedia/commons/thumb/6/64/A_spinner_dolphin_in_the_Red_Sea.jpg
ERROR:downloader:Response status code 403, file https://get.pxhere.com/photo/sea-water-nature-ocean-play-wave-animal-wildlife-wild-swim-splash-biology-mammal-blue-aquatic-flipper-life-fin-waves-marine-mammals-vertebrate-dolphin-pacific-dolphins-intelligent-marine-mammal-delphinidae-marine-biology-spinner-dolphin-whales-dolphins-and-porpoises-stenella-common-bottlenose-dolphin-short-beaked-common-dolphin-rough-toothed-dolphin-striped-dolphin-tucuxi-617045.jpg
ERROR:downloader:Response status code 403, file https://get.pxhere.com/photo/wildlife-biology-mammal-fauna-mountains-newzealand-waterfalls-vertebrate-dolphin-fiordlandnationialpark-dolphins-boatride-grandpacifictours-jucycruises-highcliffs-lunchcruise-marine-mamm


>>> [4/5] Processing and converting images...

Dataset creation completed!
Storage location: /content/drive/MyDrive/Datasets/auto_test_set/clean_224
Total valid images: 150


In [14]:
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.datasets as datasets

class ExactIndexDataset(Dataset):
    def __init__(self, root_dir, cifar_classes, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.cifar_classes = cifar_classes
        self.samples = []

        if not os.path.exists(root_dir):
            raise RuntimeError(f"Directory does not exist: {root_dir}")

        for class_name in os.listdir(root_dir):
            class_path = os.path.join(root_dir, class_name)
            if os.path.isdir(class_path):
                if class_name in self.cifar_classes:
                    real_index = self.cifar_classes.index(class_name)
                    for img_name in os.listdir(class_path):
                        if img_name.lower().endswith(('.jpg', '.jpeg')):
                            self.samples.append((os.path.join(class_path, img_name), real_index))
                else:
                    print(f"Warning: Folder '{class_name}' is not a CIFAR-100 class, ignored.")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        image = Image.open(path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label

def get_verified_test_loader(data_root, batch_size=32):
    transform_eval = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    temp_set = datasets.CIFAR100(root='./temp_data_verify', train=False, download=True)
    full_classes = temp_set.classes

    custom_dataset = ExactIndexDataset(
        root_dir=data_root,
        cifar_classes=full_classes,
        transform=transform_eval
    )

    if len(custom_dataset) == 0:
        print("Error: Dataset is empty! Please check path or folder names.")
        return None, None

    print(f"Test set built successfully: {len(custom_dataset)} images")
    print(f"   (Automatically aligned with CIFAR-100 indices and applied ImageNet Normalization)")

    loader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=False)
    return loader, full_classes

if __name__ == "__main__":
    test_path = "/content/drive/MyDrive/Datasets/auto_test_set/clean_224"
    homemade_loader, classes_list = get_verified_test_loader(test_path)

    if homemade_loader:
        print("DataLoader ready, can be passed directly to evaluation function!")

        images, labels = next(iter(homemade_loader))
        print(f"Batch Shape: {images.shape}")
        print(f"Batch Labels: {labels}")
        print(f"Corresponding class names: {[classes_list[i] for i in labels[:5]]}")

100%|██████████| 169M/169M [00:13<00:00, 12.5MB/s]


Test set built successfully: 150 images
   (Automatically aligned with CIFAR-100 indices and applied ImageNet Normalization)
DataLoader ready, can be passed directly to evaluation function!
Batch Shape: torch.Size([32, 3, 224, 224])
Batch Labels: tensor([82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 91, 91, 91,
        91, 91, 91, 91, 91, 91, 91, 91, 91, 91, 91, 91, 25, 25])
Corresponding class names: ['sunflower', 'sunflower', 'sunflower', 'sunflower', 'sunflower']


In [ ]:
from sklearn.metrics import classification_report, accuracy_score
import numpy as np
import torch

def evaluate_resnet_performance(model, dataloader, classes=None):

    model.eval()
    all_preds = []
    all_targets = []

    print(f">>> Evaluating custom test set...")

    with torch.no_grad():
        for inputs, targets in dataloader:
            inputs = inputs.to(device)
            targets = targets.to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)

            all_preds.extend(predicted.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())

    acc = accuracy_score(all_targets, all_preds)
    print("\n" + "="*50)
    print(f"Custom Test Set Performance")
    print("="*50)
    print(f"Subset Accuracy: {acc:.2%}")
    print("-" * 50)



    if classes is None:
        print("Error: Must provide classes list to generate detailed report")
        return


    all_indices = list(range(len(classes)))


    report = classification_report(
        all_targets,
        all_preds,
        labels=all_indices,
        target_names=classes,
        output_dict=True,
        zero_division=0
    )


    print(f"{'Class Name':<20} | {'Precision':<10} | {'Recall':<10} | {'F1-Score':<10} | {'Count':<5}")
    print("-" * 65)

    found_any = False
    for class_name in classes:
        if class_name in report:
            stats = report[class_name]
            if stats['support'] > 0:
                found_any = True
                print(f"{class_name:<20} | {stats['precision']:.2%}    | {stats['recall']:.2%}    | {stats['f1-score']:.2%}    | {int(stats['support']):<5}")

    if not found_any:
        print("Warning: Report generated but no known classes found in data. Please check mapping logic.")

    print("-" * 65)

    weighted = report['weighted avg']
    print(f"Weighted F1 (for current data only): {weighted['f1-score']:.2%}")
evaluate_resnet_performance(net, homemade_loader, classes=classes_list)

>>> Evaluating custom test set...

Custom Test Set Performance
Subset Accuracy: 55.33%
--------------------------------------------------
Class Name           | Precision  | Recall     | F1-Score   | Count
-----------------------------------------------------------------
couch                | 100.00%    | 93.33%    | 96.55%    | 15   
dolphin              | 100.00%    | 53.33%    | 69.57%    | 15   
lamp                 | 92.31%    | 80.00%    | 85.71%    | 15   
maple_tree           | 0.00%    | 0.00%    | 0.00%    | 15   
mouse                | 100.00%    | 20.00%    | 33.33%    | 15   
mushroom             | 100.00%    | 73.33%    | 84.62%    | 15   
rocket               | 100.00%    | 33.33%    | 50.00%    | 15   
streetcar            | 100.00%    | 60.00%    | 75.00%    | 15   
sunflower            | 83.33%    | 100.00%    | 90.91%    | 15   
trout                | 100.00%    | 40.00%    | 57.14%    | 15   
-----------------------------------------------------------------
Weighte

Dual-Channel Data-loding Pipeline  + Final Test Result

In [15]:
!pip install icrawler

import os
import shutil
import random
import uuid
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from icrawler.builtin import BingImageCrawler
from google.colab import drive
from sklearn.metrics import accuracy_score, classification_report

print(">>> [1/5] Mounting Google Drive...")
drive.mount('/content/drive')

BASE_ROOT = "/content/drive/MyDrive/Datasets/final_test_pipeline"
RAW_ROOT = os.path.join(BASE_ROOT, "raw_temp")
DIR_32   = os.path.join(BASE_ROOT, "clean_32")
DIR_224  = os.path.join(BASE_ROOT, "clean_224")

os.makedirs(RAW_ROOT, exist_ok=True)
os.makedirs(DIR_32, exist_ok=True)
os.makedirs(DIR_224, exist_ok=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Environment ready. Working directory: {BASE_ROOT}")

>>> [1/5] Mounting Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Environment ready. Working directory: /content/drive/MyDrive/Datasets/final_test_pipeline


In [16]:
def execute_crawling_and_processing(num_classes=10, images_per_class=20):
    print(f"\n>>> [2/5] Randomly selecting {num_classes} CIFAR-100 classes...")
    temp_set = datasets.CIFAR100(root='./temp_meta', train=False, download=True)
    cifar_classes = temp_set.classes

    selected_keywords = random.sample(cifar_classes, num_classes)
    print(f"Selected targets: {selected_keywords}")

    for keyword in selected_keywords:
        print(f"Crawling: {keyword}...")
        save_dir = os.path.join(RAW_ROOT, keyword)
        os.makedirs(save_dir, exist_ok=True)

        crawler = BingImageCrawler(
            downloader_threads=4,
            storage={"root_dir": save_dir},
            log_level="ERROR"
        )
        crawler.crawl(keyword=f"{keyword} photo", max_num=images_per_class)

    print("\n>>> [3/5] Generating dual-version dataset (32px & 224px)...")
    valid_exts = {".jpg", ".jpeg", ".png", ".bmp", ".webp"}
    count = 0

    for class_name in os.listdir(RAW_ROOT):
        class_dir = os.path.join(RAW_ROOT, class_name)
        if not os.path.isdir(class_dir): continue

        os.makedirs(os.path.join(DIR_32, class_name), exist_ok=True)
        os.makedirs(os.path.join(DIR_224, class_name), exist_ok=True)

        for filename in os.listdir(class_dir):
            if not any(filename.lower().endswith(ext) for ext in valid_exts): continue

            try:
                with Image.open(os.path.join(class_dir, filename)) as img:
                    img = img.convert("RGB")
                    unique_name = f"{class_name}_{uuid.uuid4().hex[:8]}.jpg"

                    img.resize((32, 32), Image.LANCZOS).save(
                        os.path.join(DIR_32, class_name, unique_name), "JPEG", quality=95
                    )

                    img.resize((224, 224), Image.BICUBIC).save(
                        os.path.join(DIR_224, class_name, unique_name), "JPEG", quality=95
                    )
                    count += 1
            except Exception:
                pass

    print(f"Processing completed! Generated {count} pairs of test images.")
    shutil.rmtree(RAW_ROOT)

In [17]:
class ExactIndexDataset(Dataset):
    def __init__(self, root_dir, cifar_classes, transform=None):
        self.samples = []
        self.transform = transform

        if not os.path.exists(root_dir): return

        for class_name in os.listdir(root_dir):
            class_path = os.path.join(root_dir, class_name)
            if os.path.isdir(class_path) and class_name in cifar_classes:
                real_index = cifar_classes.index(class_name)
                for img in os.listdir(class_path):
                    if img.endswith('.jpg'):
                        self.samples.append((os.path.join(class_path, img), real_index))

    def __len__(self): return len(self.samples)
    def __getitem__(self, idx):
        path, label = self.samples[idx]
        img = Image.open(path).convert('RGB')
        if self.transform: img = self.transform(img)
        return img, label

def get_universal_loader(model_type, batch_size=32):
    if model_type == 'cifar_32':
        mean, std = [0.507, 0.487, 0.441], [0.267, 0.256, 0.276]
        data_path = DIR_32
    elif model_type == 'imagenet_224':
        mean, std = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]
        data_path = DIR_224
    else:
        raise ValueError("Unknown model type")

    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])

    full_classes = datasets.CIFAR100(root='./temp_meta', train=False).classes
    dataset = ExactIndexDataset(data_path, full_classes, transform)

    if len(dataset) == 0:
        print(f"Warning: No data found in {model_type} path! Please run crawling step first.")
        return None, full_classes

    return DataLoader(dataset, batch_size=batch_size, shuffle=False), full_classes

def evaluate_model(model, dataloader, classes):
    model.eval()
    all_preds, all_targets = [], []

    with torch.no_grad():
        for inputs, targets in dataloader:
            inputs, targets = inputs.to(device), targets.to(device)
            _, predicted = torch.max(model(inputs), 1)
            all_preds.extend(predicted.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())

    report = classification_report(
        all_targets, all_preds,
        labels=list(range(100)), target_names=classes,
        output_dict=True, zero_division=0
    )

    print(f"Subset Accuracy: {accuracy_score(all_targets, all_preds):.2%}")
    print(f"{'Class':<15} | {'Prec.':<8} | {'Recall':<8} | {'Count'}")
    print("-" * 45)
    for name in classes:
        if name in report and report[name]['support'] > 0:
            r = report[name]
            print(f"{name:<15} | {r['precision']:.1%} | {r['recall']:.1%} | {int(r['support'])}")

In [18]:
# Fetch & Pre-processing data
execute_crawling_and_processing(num_classes=10, images_per_class=15)


>>> [2/5] Randomly selecting 10 CIFAR-100 classes...


100%|██████████| 169M/169M [00:14<00:00, 11.8MB/s]


Selected targets: ['can', 'hamster', 'kangaroo', 'possum', 'lion', 'mountain', 'orchid', 'shark', 'maple_tree', 'mushroom']
Crawling: can...


ERROR:downloader:Response status code 400, file https://media.istockphoto.com/id/863873832/photo/close-up-view-of-can.jpg
ERROR:downloader:Response status code 404, file https://i0.wp.com/onlearn.es/wp-content/uploads/2023/03/can-grammar.jpg
ERROR:downloader:Response status code 400, file https://media.istockphoto.com/id/697218800/photo/an-image-of-can.jpg


Crawling: hamster...


...

Crawling: shark...


ERROR:downloader:Response status code 403, file https://i1.pickpik.com/photos/94/786/386/shark-fish-eye-animal-thumb.jpg
ERROR:downloader:Response status code 403, file https://c.pxhere.com/photos/1a/05/shark_teeth_underwater_sea_predator_jaw-942109.jpg
ERROR:downloader:Response status code 403, file https://img1.whatsthatfish.com/unsafe/810x540/image.whatsthatfish.com/original/mEynI79zMe.jpg
ERROR:downloader:Response status code 403, file https://c.pxhere.com/photos/c6/0a/shark_ocean_sea_water_animal_predator_nature_life-1334532.jpg
ERROR:downloader:Response status code 403, file https://get.pxhere.com/photo/biology-blue-fish-shark-marine-vertebrate-marine-biology-requiem-shark-carcharhiniformes-tiger-shark-cartilaginous-fish-565928.jpg


Crawling: maple_tree...
Crawling: mushroom...


ERROR:downloader:Response status code 403, file https://get.pxhere.com/photo/nature-forest-flower-autumn-mushroom-close-forest-floor-fungus-mushrooms-toxic-agaric-bolete-fungal-species-forest-mushroom-lamellar-matsutake-matryoshka-forest-plant-symbol-of-good-luck-screen-fungus-edible-mushroom-medicinal-mushroom-agaricaceae-penny-bun-603884.jpg
ERROR:downloader:Response status code 403, file https://get.pxhere.com/photo/nature-forest-meadow-prairie-flower-gift-red-autumn-mushroom-point-flora-fungus-spotted-fly-agaric-toxic-amanita-agaric-bolete-amanita-muscaria-muscaria-red-fly-agaric-mushroom-macro-photography-edible-mushroom-medicinal-mushroom-penny-bun-amanitaceae-1157240.jpg
ERROR:downloader:Response status code 403, file https://get.pxhere.com/photo/nature-forest-flower-autumn-botany-mushroom-flora-fauna-fungus-mushrooms-woodland-agaric-bolete-agaricus-matsutake-oyster-mushroom-edible-mushroom-medicinal-mushroom-agaricomycetes-agaricaceae-champignon-mushroom-penny-bun-pleurotus-ery


>>> [3/5] Generating dual-version dataset (32px & 224px)...
Processing completed! Generated 146 pairs of test images.


In [19]:
print("\nTesting 32x32 model (CIFAR-100 Scratch)...")

loader_32, classes = get_universal_loader(model_type='cifar_32')

if loader_32:
    evaluate_model(net1, loader_32, classes)


Testing 32x32 model (CIFAR-100 Scratch)...
Subset Accuracy: 38.54%
Class           | Prec.    | Recall   | Count
---------------------------------------------
bed             | 100.0% | 30.0% | 10
can             | 100.0% | 46.7% | 15
flatfish        | 100.0% | 6.7% | 15
girl            | 100.0% | 33.3% | 15
hamster         | 100.0% | 26.7% | 15
kangaroo        | 100.0% | 9.1% | 11
lion            | 84.6% | 73.3% | 15
lobster         | 66.7% | 40.0% | 15
maple_tree      | 0.0% | 0.0% | 15
mountain        | 100.0% | 53.3% | 15
mouse           | 50.0% | 6.7% | 15
mushroom        | 100.0% | 66.7% | 15
orchid          | 81.8% | 60.0% | 15
plate           | 100.0% | 40.0% | 15
possum          | 85.7% | 40.0% | 15
rabbit          | 100.0% | 13.3% | 15
rose            | 100.0% | 60.0% | 15
shark           | 100.0% | 40.0% | 15
skyscraper      | 100.0% | 46.7% | 15
tank            | 90.0% | 75.0% | 12


In [20]:
print("\nTesting 224x224 model (ImageNet Transfer)...")

loader_224, classes = get_universal_loader(model_type='imagenet_224')

if loader_224:
    evaluate_model(net, loader_224, classes)


Testing 224x224 model (ImageNet Transfer)...
Subset Accuracy: 56.94%
Class           | Prec.    | Recall   | Count
---------------------------------------------
bed             | 100.0% | 40.0% | 10
can             | 100.0% | 33.3% | 15
flatfish        | 100.0% | 20.0% | 15
girl            | 100.0% | 6.7% | 15
hamster         | 100.0% | 26.7% | 15
kangaroo        | 100.0% | 81.8% | 11
lion            | 100.0% | 80.0% | 15
lobster         | 90.9% | 66.7% | 15
maple_tree      | 0.0% | 0.0% | 15
mountain        | 100.0% | 60.0% | 15
mouse           | 20.0% | 13.3% | 15
mushroom        | 100.0% | 73.3% | 15
orchid          | 77.8% | 93.3% | 15
plate           | 100.0% | 53.3% | 15
possum          | 100.0% | 100.0% | 15
rabbit          | 86.7% | 86.7% | 15
rose            | 100.0% | 86.7% | 15
shark           | 100.0% | 73.3% | 15
skyscraper      | 100.0% | 66.7% | 15
tank            | 100.0% | 83.3% | 12
